In [1]:
# imports
import tensorflow as tf

In [7]:
# We have designed a class to stop the training when the model reaches a desired accuracy

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.998):
            print("\nReached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True
                
callbacks = myCallback()


In [8]:
# Load the mnist dataset
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data() 

In [3]:
# First we will reshape the images and adjust the values to have a faster model
# That's because the first convolution expects a single tensor containing everything, 
# so instead of 60,000 28x28x1 items in a list, we have a single 4D list that is 60,000x28x28x1, 
# and the same for the test images. 
# If you don't do this, you'll get an error when training as the Convolutions do not recognize the shape. 

training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0


In [4]:
# Model with 3 convilutional layers, each followed by a maxpooling layer.
# The model is then followed by the previous neural network

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()    # To see the steps
model.fit(training_images, training_labels, epochs=20, callbacks=[callbacks])
test_loss = model.evaluate(test_images, test_labels)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               204928    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [6]:
# Now we want to do a little prediction to see if it can say what a random picture from the test set is!

classifications = model.predict(test_images)

print(classifications[10]*100)
print(test_labels[10])

[5.1624735e-04 2.6852867e-06 4.8101287e-02 1.4629580e-07 9.9626717e+01
 6.8388562e-07 3.2465321e-01 2.6918068e-08 2.9954805e-07 9.5582673e-06]
4
